<a href="https://colab.research.google.com/github/Nageswar-Sahoo/Computer-Vision-Project/blob/main/S7/traininglogsincoolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b  backup-1  https://github.com/Nageswar-Sahoo/Computer-Vision-Project.git

Cloning into 'Computer-Vision-Project'...
remote: Enumerating objects: 577, done.
remote: Counting objects: 100% (577/577), done.
remote: Compressing objects: 100% (498/498), done.
remote: Total 577 (delta 362), reused 203 (delta 67), pack-reused 0
Receiving objects: 100% (577/577), 1.87 MiB | 14.37 MiB/s, done.
Resolving deltas: 100% (362/362), done.


In [2]:
!pip install albumentations==0.4.6
import albumentations 
from albumentations.pytorch import ToTensorV2

     |████████████████████████████████| 117 kB 5.3 MB/s 
     |████████████████████████████████| 948 kB 36.2 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=58f0f2e113b7d99954db45cd13b29b91c9ac26024aec67aa3ad2f785147c7439
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [3]:
cd Computer-Vision-Project/S7

/content/Computer-Vision-Project/S7


In [25]:
rm -rf Computer-Vision-Project

In [4]:
cat model/model.py

import torch.nn as nn
import torch.nn.functional as F
from base import BaseModel


class CIFRModel(BaseModel):
    # epoch: 8
    # loss: 0.73457906589928
    # accuracy: 75.11097301136364
    # val_loss: 0.7309541015685359
    # val_accuracy: 74.72310126582279

    def __init__(self, num_classes=10, normalizationtype='Layer', dropout=.01):
        super().__init__()
        # Input Block

        self.depthwise_separable_conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, groups=3, padding=1, bias=False),
            nn.BatchNorm2d(3),
            # nn.Dropout(.05),
            nn.ReLU(),
            nn.Conv2d(in_channels=3, out_channels=256, kernel_size=1, bias=False),
            nn.BatchNorm2d(256),
            # nn.Dropout(.05),
            nn.ReLU(),
        )  # output_size = 28 receptive field  : 5

        # Input Block
        self.depthwise_separable_conv2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_

In [5]:
import data_loader.data_loaders as module_data
from parse_config import ConfigParser
from trainer import Trainer
from utils import prepare_device
import torch
import numpy as np
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import logging

# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [ ]:
cat config.json

In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = module_arch.CIFRModel().to(device)
summary(model, input_size=(3, 32, 32))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 32, 32]              27
       BatchNorm2d-2            [-1, 3, 32, 32]               6
              ReLU-3            [-1, 3, 32, 32]               0
            Conv2d-4          [-1, 256, 32, 32]             768
       BatchNorm2d-5          [-1, 256, 32, 32]             512
              ReLU-6          [-1, 256, 32, 32]               0
            Conv2d-7          [-1, 256, 32, 32]           2,304
       BatchNorm2d-8          [-1, 256, 32, 32]             512
              ReLU-9          [-1, 256, 32, 32]               0
           Conv2d-10           [-1, 40, 32, 32]          10,240
      BatchNorm2d-11           [-1, 40, 32, 32]              80
             ReLU-12           [-1, 40, 32, 32]               0
  ConvTranspose2d-13          [-1, 185, 48, 48]          66,600
      BatchNorm2d-14          [-1, 

In [8]:
    logger = logging.getLogger("trian")
    # Read the config.json
    config = ConfigParser.from_args()

    # setup data_loader instances
    data_loader = module_data.CIFRDataLoader(data_dir='data/', batch_size=64, shuffle=True, validation_split=0.1,
                                             num_workers=2, training=True)
    valid_data_loader = data_loader.split_validation()

    # build model architecture, then print to console
    model = module_arch.CIFRModel()
    logger.info(model)

    # prepare for (multi-device) GPU training
    n_gpu = 1
    device, device_ids = prepare_device(n_gpu)
    model = model.to(device)
    if len(device_ids) > 1:
        model = torch.nn.DataParallel(model, device_ids=device_ids)

    # get function handles of loss and metrics
    criterion = module_loss.crossentropyloss
    metrics = [module_metric.accuracy]

    # build optimizer, learning rate scheduler. delete every lines containing lr_scheduler for disabling scheduler
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    #optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    lr_scheduler = StepLR(optimizer, step_size=80, gamma=0.1)
    trainer = Trainer(model, criterion, metrics, optimizer,
                      config=config,
                      device=device,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                      lr_scheduler=lr_scheduler)


In [20]:
trainer.train()

    epoch          : 1
    loss           : 1.4654897752810607
    accuracy       : 48.30211292613637
    val_loss       : 1.3524107933044434
    val_accuracy   : 51.859177215189874
Saving checkpoint: saved/models/CIFR10/1115_031256/checkpoint-epoch1.pth ...
Saving current best: model_best.pth ...
    epoch          : 2
    loss           : 1.1102368023923852
    accuracy       : 61.47238991477273
    val_loss       : 1.112451823451851
    val_accuracy   : 60.71993670886076
Saving checkpoint: saved/models/CIFR10/1115_031256/checkpoint-epoch2.pth ...
Saving current best: model_best.pth ...
    epoch          : 3
    loss           : 1.0027489319274372
    accuracy       : 65.16557173295455
    val_loss       : 0.9853062305269362
    val_accuracy   : 64.2998417721519
Saving checkpoint: saved/models/CIFR10/1115_031256/checkpoint-epoch3.pth ...
Saving current best: model_best.pth ...
    epoch          : 4
    loss           : 0.9436435674909841
    accuracy       : 67.32066761363636
    v

KeyboardInterrupt: ignored